In [1]:
#%% Paquetes a utilizar
import re
import time
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode
from nltk.stem import SnowballStemmer

In [2]:
#%% Carga de catálogos
catalogo = pd.read_csv('/home/milo/Documents/Python/TextMining/catalogo.csv', encoding='ISO-8859-1').\
           fillna('   ')
# Cargamos la base a calificar
#%% Cargar base a calificar
base = pd.read_csv('/home/milo/Documents/Python/TextMining/CalifConeval.csv',
                     encoding='ISO-8859-1',
                     index_col='RMP').fillna("")

In [3]:
catalogo.shape

(1229, 7)

In [4]:
# Generamos una función que aplicará las transformaciones necesarias al texto

def nltk_fun(column):
  # Cambiamos mayúsculas por minúsculas
  
  column=column.apply(lambda x: " ".join(x.lower() for x in x.split()))
  # Removemos Stopwords
  import nltk
  from nltk.corpus import stopwords
  
  stop = stopwords.words('spanish')
  
  column=column.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
  # Removemos puntuación
  import re
  
  column=column.str.replace(r'[^\w\s]', '')
  # Removemos acentos
  import unidecode
  
  column=column.apply(lambda x: "".join(unidecode.unidecode(x)))
  # Tokenizing Text
  column=column.apply(lambda x: nltk.word_tokenize(x))
  # Realizamos el stemming
  stemmer = SnowballStemmer("spanish")
  column=column.apply(lambda x: [stemmer.stem(y) for y in x])
  
  # Cambiamos las listas a String
  column=column.apply(lambda x: " ".join(str(y) for y in x))

  # Eliminamos los espacios innecesarios
  column=column.apply(lambda x: re.sub(' +',' ',x))
  
  return(column)

In [5]:
### Aplicamos la función anterior a cada una de las columnas en el catálogo y en las descripciones

base_lista=['DESC_FIN','DESC_PROPOSITO','DESC_PP','DESC_AI']
catalogo_lista=['Categoria','Categoria_Auxiliar_1','Categoria_Auxiliar_2']


for x in base_lista:
    base[x]=nltk_fun(base[x])
    
for x in catalogo_lista:
    catalogo[x]=nltk_fun(catalogo[x])

In [6]:
catalogo.head()

,id,Objetivo,id_derecho,Derecho,Categoria,Categoria_Auxiliar_1,Categoria_Auxiliar_2
0,1.1,1,1,Alimentación,acuicol,comerci,
1,1.1,1,1,Alimentación,acuicol,produccion,
2,1.1,1,1,Alimentación,acuicol,product,
3,1.1,1,1,Alimentación,acuicol,productor,
4,1.1,1,1,Alimentación,agricol,comerci,


In [7]:
base.head()

,Ciclo,DESC_FIN,DESC_PROPOSITO,DESC_PP,DESC_AI
RMP,,,,,
1K025,2008,,,proyect inmuebl oficin administr,llev cab proces legisl
1K027,2008,,,manten infraestructur,llev cab proces legisl
1R001,2008,,,activ deriv trabaj legisl,llev cab proces legisl
1R002,2008,,,present inform result,fiscalizacion cuent haciend public federal
1R301,2008,,,activ deriv trabaj legisl ampliacion determin ...,llev cab proces legisl


In [8]:
### Cambiamos algunas palabras a  cada una de las columnas en el catálogo y en las descripciones

lista_uno=["\\bagu\\b","\\bcomun\\b","\\bcontingent\\b","\\bguard\\b","\\bpol\\b","\\bsostenibil\\b","\\bzon\\b",
           "\\bmicr\\b","\\bpequeñ\\b","\\bmedian\\b","\\bcooper\\b","\\basistent\\b",
           "\\bcalid\\b","\\bcomun\\b","\\bpermanent\\b","\\bproduct\\b","\\bsegur\\b","\\bfinanci\\b","\\bfinanci\\b"]
lista_dos=['agua','comunidad','contingencia','guarderia','polo','sostenib','zona','micro','peque','mediana',
           'cooperativa','asistencia','calidad','comunic','perman','productividad','seguridad','financiamiento','financiamiento']


import time
start = time.time()

for y in range(0,18):
    for x in catalogo_lista:
        catalogo[x]=catalogo[x].apply(lambda z: re.sub(lista_uno[y],lista_dos[y],z))
        # Cambiamos las listas a String
        catalogo[x]=catalogo[x].apply(lambda z: "".join(str(y) for y in z))


for y in range(0,18):
    for x in base_lista:
        base[x]=base[x].apply(lambda z: re.sub(lista_uno[y],lista_dos[y],z))
        # Cambiamos las listas a String
        base[x]=base[x].apply(lambda z: "".join(str(y) for y in z))

end = time.time()
print(end - start)



8.163651704788208


## Aquí tenemos dos opciones para aplicar la búsqueda: 

1. La primera nos dará toda la cadena de coincidencia (i.e. agregará tanto la característica principal como las auxiliares).
1. La segunda nos quitará los valores repetidos en las cadenas


### Opción 1

In [9]:
# Generamos una función que aplicará la búsqueda de la opción 1

def busqueda_uno(column):

    descri=base[[column]]
    descri['Coincidencias']=""
    descri['ObjeDereSocial']=""

    # Hacemos un for para quedarnos con las filas de DESC_FIN que cumplen con alguna de las características del catálogo

    for x in range(0,1228):
    
        target=[catalogo.iloc[x,4],catalogo.iloc[x,5],catalogo.iloc[x,6]]
        index=descri[column].apply(lambda x: all(word in x for word in target))
        list_index=index.reset_index()[index.reset_index()[column]==True].index.values.tolist()
        if descri.iloc[list_index,1].empty:
            pass
        else:
            coinci=[descri.iloc[list_index,1].iloc[0],";",catalogo.iloc[x,1],catalogo.iloc[x,2],"p",catalogo.iloc[x,4],
                  catalogo.iloc[x,1],catalogo.iloc[x,2],"a1",catalogo.iloc[x,5],catalogo.iloc[x,1],catalogo.iloc[x,2],
                    "a2",catalogo.iloc[x,6]]
            text="-".join(str(y) for y in coinci)
            descri.iloc[list_index,1]=text
        
            dereSoc=[catalogo.iloc[x,3]]
            descri.iloc[list_index,2]=descri.iloc[list_index,2]+";"+";".join(str(y) for y in dereSoc)

    descri['Coincidencias']=descri['Coincidencias'].apply(lambda z: "|".join(str(y) for y in set(z.split(";"))))        
    descri['ObjeDereSocial']=descri['ObjeDereSocial'].apply(lambda z: ",".join(str(y) for y in set(z.split(";"))))
    
    return(descri)

In [10]:
# Generamos una función que aplicará la búsqueda de la opción 2

def busqueda_dos(column):
    
    descri=base[[column]]
    descri['Coincidencias']=""
    descri['ObjeDereSocial']=""

    # Hacemos un for para quedarnos con las filas de DESC_FIN que cumplen con alguna de las características del catálogo

    for x in range(0,1228):
    
        target=[catalogo.iloc[x,4],catalogo.iloc[x,5],catalogo.iloc[x,6]]
        index=descri[column].apply(lambda x: all(word in x for word in target))
        list_index=index.reset_index()[index.reset_index()[column]==True].index.values.tolist()
        if descri.iloc[list_index,1].empty:
            pass
        else:
            coinci=[descri.iloc[list_index,1].iloc[0],";",catalogo.iloc[x,1],catalogo.iloc[x,2],"p",catalogo.iloc[x,4],";",
                  catalogo.iloc[x,1],catalogo.iloc[x,2],"a1",catalogo.iloc[x,5],";",catalogo.iloc[x,1],catalogo.iloc[x,2],
                    "a2",catalogo.iloc[x,6]]
            text="-".join(str(y) for y in coinci)
            descri.iloc[list_index,1]=text
        
            dereSoc=[catalogo.iloc[x,3]]
            descri.iloc[list_index,2]=descri.iloc[list_index,2]+";"+";".join(str(y) for y in dereSoc)

    descri['Coincidencias']=descri['Coincidencias'].apply(lambda z: "|".join(str(y) for y in set(z.split(";"))))        
    descri['ObjeDereSocial']=descri['ObjeDereSocial'].apply(lambda z: ",".join(str(y) for y in set(z.split(";"))))

    return(descri)

                
        

In [11]:
import time
start = time.time()

df_bus_uno_desc_fin=busqueda_uno('DESC_FIN')
df_bus_dos_desc_fin=busqueda_dos('DESC_FIN')

df_bus_uno_desc_prop=busqueda_uno('DESC_PROPOSITO')
df_bus_dos_desc_prop=busqueda_dos('DESC_PROPOSITO')

df_bus_uno_desc_pp=busqueda_uno('DESC_PP')
df_bus_dos_desc_pp=busqueda_dos('DESC_PP')

df_bus_uno_desc_ai=busqueda_uno('DESC_AI')
df_bus_dos_desc_ai=busqueda_dos('DESC_AI')

end = time.time()
print(end - start)


/home/milo/miniconda3/envs/cursoap/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/milo/miniconda3/envs/cursoap/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/milo/miniconda3/envs/cursoap/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://panda

121.55798387527466


In [37]:
desc_fin = pd.concat([df_bus_uno_desc_fin, df_bus_dos_desc_fin[['Coincidencias','ObjeDereSocial']]], axis=1)
desc_prop = pd.concat([df_bus_uno_desc_prop, df_bus_dos_desc_prop[['Coincidencias','ObjeDereSocial']]], axis=1)
desc_pp = pd.concat([df_bus_uno_desc_pp, df_bus_dos_desc_pp[['Coincidencias','ObjeDereSocial']]], axis=1)
desc_ai = pd.concat([df_bus_uno_desc_ai, df_bus_dos_desc_ai[['Coincidencias','ObjeDereSocial']]], axis=1)

total=pd.concat([desc_fin,desc_prop,desc_pp,desc_ai],axis=1)

In [64]:
desc_fin.to_csv(r'C:/Users/End User/Desktop/TextMining/calif_desc_fin.csv')
desc_prop.to_csv(r'C:/Users/End User/Desktop/TextMining/calif_desc_prop.csv')
desc_pp.to_csv(r'C:/Users/End User/Desktop/TextMining/calif_desc_pp.csv')
desc_ai.to_csv(r'C:/Users/End User/Desktop/TextMining/calif_desc_ai.csv')